In [1]:
from functools import partial

import os
import torch
import numpy as np
import torch.nn as nn
import pandas as pd
import matplotlib.pyplot as plt
import math
import datetime

from tabulate import tabulate
from timm.models.vision_transformer import Block
# from utils import MaskEmbed, get_1d_sincos_pos_embed, ActiveEmbed, Utils

eps = 1e-6

In [2]:
import wandb

# Log in to your W&B account (if not already logged in)
wandb.login()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: aneog. Use `wandb login --relogin` to force relogin


True

In [3]:
api = wandb.Api()

In [4]:
runs = api.runs("aneog/ett")
list_of_runs = []
for run in runs:
    list_of_runs.append((run.id, run.name))
    print(f"Run ID: {run.id}, Name: {run.name}")

Run ID: 2mr6t21i, Name: finetune_v1_finetune_ETTh1_mask_75_NOT_FRZN_2024-03-06_03:07:15.656449
Run ID: 9wk2osze, Name: finetune_v1_finetune_ETTm2_mask_75_NOT_FRZN_2024-03-06_02:55:31.803292
Run ID: 27escbdo, Name: finetune_v1_finetune_ETTh1_mask_50_NOT_FRZN_2024-03-06_02:50:59.879849
Run ID: t1hpy3i5, Name: finetune_v1_finetune_ETTm2_mask_50_NOT_FRZN_2024-03-06_02:41:51.728101
Run ID: hz4grdda, Name: finetune_v1_finetune_ETTh1_mask_25_NOT_FRZN_2024-03-06_02:30:02.259881
Run ID: 24qaofj2, Name: finetune_v1_finetune_ETTm2_mask_25_NOT_FRZN_2024-03-06_02:28:09.632661
Run ID: 795ruzmk, Name: finetune_v1_finetune_ETTm2_mask_75_FRZN_2024-03-06_02:15:09.575722
Run ID: 30q7rqij, Name: finetune_v1_finetune_ETTh2_mask_75_NOT_FRZN_2024-03-06_02:09:53.986500
Run ID: 3jeq1g35, Name: finetune_v1_finetune_ETTm2_mask_50_FRZN_2024-03-06_02:02:03.669746
Run ID: 1qerrp3s, Name: finetune_v1_finetune_ETTh1_mask_75_FRZN_2024-03-06_02:01:58.181763
Run ID: j9ly5app, Name: finetune_v1_finetune_ETTm1_mask_75_NOT

In [5]:
date_regex_a = "2024-02-20"
date_regex_b = "2024-02-21"

In [6]:
filtered_runs =  [(runid, name) for runid, name in list_of_runs]# if date_regex_a in name or date_regex_b in name]

In [7]:
filtered_runs

[('2mr6t21i',
  'finetune_v1_finetune_ETTh1_mask_75_NOT_FRZN_2024-03-06_03:07:15.656449'),
 ('9wk2osze',
  'finetune_v1_finetune_ETTm2_mask_75_NOT_FRZN_2024-03-06_02:55:31.803292'),
 ('27escbdo',
  'finetune_v1_finetune_ETTh1_mask_50_NOT_FRZN_2024-03-06_02:50:59.879849'),
 ('t1hpy3i5',
  'finetune_v1_finetune_ETTm2_mask_50_NOT_FRZN_2024-03-06_02:41:51.728101'),
 ('hz4grdda',
  'finetune_v1_finetune_ETTh1_mask_25_NOT_FRZN_2024-03-06_02:30:02.259881'),
 ('24qaofj2',
  'finetune_v1_finetune_ETTm2_mask_25_NOT_FRZN_2024-03-06_02:28:09.632661'),
 ('795ruzmk',
  'finetune_v1_finetune_ETTm2_mask_75_FRZN_2024-03-06_02:15:09.575722'),
 ('30q7rqij',
  'finetune_v1_finetune_ETTh2_mask_75_NOT_FRZN_2024-03-06_02:09:53.986500'),
 ('3jeq1g35',
  'finetune_v1_finetune_ETTm2_mask_50_FRZN_2024-03-06_02:02:03.669746'),
 ('1qerrp3s',
  'finetune_v1_finetune_ETTh1_mask_75_FRZN_2024-03-06_02:01:58.181763'),
 ('j9ly5app',
  'finetune_v1_finetune_ETTm1_mask_75_NOT_FRZN_2024-03-06_01:43:36.635241'),
 ('ygw0v3ew

In [11]:
'''
removing last couple of runs: not part of the experiments
'''
# exp_runs = filtered_runs[:-13]
exp_runs = filtered_runs

In [12]:
len(exp_runs)

36

In [13]:
exp_runs

[('2mr6t21i',
  'finetune_v1_finetune_ETTh1_mask_75_NOT_FRZN_2024-03-06_03:07:15.656449'),
 ('9wk2osze',
  'finetune_v1_finetune_ETTm2_mask_75_NOT_FRZN_2024-03-06_02:55:31.803292'),
 ('27escbdo',
  'finetune_v1_finetune_ETTh1_mask_50_NOT_FRZN_2024-03-06_02:50:59.879849'),
 ('t1hpy3i5',
  'finetune_v1_finetune_ETTm2_mask_50_NOT_FRZN_2024-03-06_02:41:51.728101'),
 ('hz4grdda',
  'finetune_v1_finetune_ETTh1_mask_25_NOT_FRZN_2024-03-06_02:30:02.259881'),
 ('24qaofj2',
  'finetune_v1_finetune_ETTm2_mask_25_NOT_FRZN_2024-03-06_02:28:09.632661'),
 ('795ruzmk',
  'finetune_v1_finetune_ETTm2_mask_75_FRZN_2024-03-06_02:15:09.575722'),
 ('30q7rqij',
  'finetune_v1_finetune_ETTh2_mask_75_NOT_FRZN_2024-03-06_02:09:53.986500'),
 ('3jeq1g35',
  'finetune_v1_finetune_ETTm2_mask_50_FRZN_2024-03-06_02:02:03.669746'),
 ('1qerrp3s',
  'finetune_v1_finetune_ETTh1_mask_75_FRZN_2024-03-06_02:01:58.181763'),
 ('j9ly5app',
  'finetune_v1_finetune_ETTm1_mask_75_NOT_FRZN_2024-03-06_01:43:36.635241'),
 ('ygw0v3ew

In [14]:
'''
sorting the list in ascending order
'''
exp_runs = exp_runs[::-1]
exp_runs

[('27r51dcb', 'pretrain_v1_pretrain_ETTh1_mask_25_2024-03-05_23:09:41.446873'),
 ('1jd1qae9', 'pretrain_v1_pretrain_ETTh2_mask_25_2024-03-05_23:09:50.208599'),
 ('1gve2zc9', 'pretrain_v1_pretrain_ETTm1_mask_25_2024-03-05_23:10:01.710980'),
 ('1pqd5l5t', 'pretrain_v1_pretrain_ETTm2_mask_25_2024-03-05_23:10:11.573805'),
 ('ejhepj4k', 'pretrain_v1_pretrain_ETTh2_mask_50_2024-03-05_23:20:15.457856'),
 ('1r0w5287', 'pretrain_v1_pretrain_ETTh1_mask_50_2024-03-05_23:24:39.183368'),
 ('13tmt4pk', 'pretrain_v1_pretrain_ETTh2_mask_75_2024-03-05_23:34:39.283505'),
 ('3ofhyukc', 'pretrain_v1_pretrain_ETTm1_mask_50_2024-03-05_23:41:47.157029'),
 ('1md57u90',
  'finetune_v1_finetune_ETTh2_mask_25_FRZN_2024-03-05_23:47:05.142523'),
 ('2malkfja', 'pretrain_v1_pretrain_ETTh1_mask_75_2024-03-05_23:52:07.764429'),
 ('173yqu26', 'pretrain_v1_pretrain_ETTm2_mask_50_2024-03-06_00:06:49.304020'),
 ('1ftvmzi8', 'pretrain_v1_pretrain_ETTm1_mask_75_2024-03-06_00:11:46.385247'),
 ('3uzq5sf5',
  'finetune_v1_fine

In [22]:
summaries = []

pretrain_val_features = ['val_rmse']
pretrain_test_features = ['test_rmse']

finetune_val_features = ['val_rmse']
finetune_train_features = ['train_rmse']
finetune_test_features = ['test_rmse']

In [15]:
'''
using the api, get all the values for the corresponding experiments
'''

'\nusing the api, get all the values for the corresponding experiments\n'

In [16]:
'''
all of these runs are in order (sorted wrt time)
'''
etth1 = [(runid, name) for runid, name in exp_runs if 'ETTh1' in name]

etth2 = [(runid, name) for runid, name in exp_runs if 'ETTh2' in name]

ettm1 = [(runid, name) for runid, name in exp_runs if 'ETTm1' in name]

ettm2 = [(runid, name) for runid, name in exp_runs if 'ETTm2' in name]

In [17]:
etth1

[('27r51dcb', 'pretrain_v1_pretrain_ETTh1_mask_25_2024-03-05_23:09:41.446873'),
 ('1r0w5287', 'pretrain_v1_pretrain_ETTh1_mask_50_2024-03-05_23:24:39.183368'),
 ('2malkfja', 'pretrain_v1_pretrain_ETTh1_mask_75_2024-03-05_23:52:07.764429'),
 ('3uzq5sf5',
  'finetune_v1_finetune_ETTh1_mask_25_FRZN_2024-03-06_00:15:16.376254'),
 ('2d0bqfb6',
  'finetune_v1_finetune_ETTh1_mask_50_FRZN_2024-03-06_01:08:01.749827'),
 ('1qerrp3s',
  'finetune_v1_finetune_ETTh1_mask_75_FRZN_2024-03-06_02:01:58.181763'),
 ('hz4grdda',
  'finetune_v1_finetune_ETTh1_mask_25_NOT_FRZN_2024-03-06_02:30:02.259881'),
 ('27escbdo',
  'finetune_v1_finetune_ETTh1_mask_50_NOT_FRZN_2024-03-06_02:50:59.879849'),
 ('2mr6t21i',
  'finetune_v1_finetune_ETTh1_mask_75_NOT_FRZN_2024-03-06_03:07:15.656449')]

In [18]:
etth2

[('1jd1qae9', 'pretrain_v1_pretrain_ETTh2_mask_25_2024-03-05_23:09:50.208599'),
 ('ejhepj4k', 'pretrain_v1_pretrain_ETTh2_mask_50_2024-03-05_23:20:15.457856'),
 ('13tmt4pk', 'pretrain_v1_pretrain_ETTh2_mask_75_2024-03-05_23:34:39.283505'),
 ('1md57u90',
  'finetune_v1_finetune_ETTh2_mask_25_FRZN_2024-03-05_23:47:05.142523'),
 ('3s1upkn9',
  'finetune_v1_finetune_ETTh2_mask_50_FRZN_2024-03-06_00:16:14.255934'),
 ('giuu4q6c',
  'finetune_v1_finetune_ETTh2_mask_75_FRZN_2024-03-06_00:44:31.044504'),
 ('jpcd6v8a',
  'finetune_v1_finetune_ETTh2_mask_25_NOT_FRZN_2024-03-06_01:13:49.788276'),
 ('ygw0v3ew',
  'finetune_v1_finetune_ETTh2_mask_50_NOT_FRZN_2024-03-06_01:42:45.253870'),
 ('30q7rqij',
  'finetune_v1_finetune_ETTh2_mask_75_NOT_FRZN_2024-03-06_02:09:53.986500')]

In [19]:
ettm1

[('1gve2zc9', 'pretrain_v1_pretrain_ETTm1_mask_25_2024-03-05_23:10:01.710980'),
 ('3ofhyukc', 'pretrain_v1_pretrain_ETTm1_mask_50_2024-03-05_23:41:47.157029'),
 ('1ftvmzi8', 'pretrain_v1_pretrain_ETTm1_mask_75_2024-03-06_00:11:46.385247'),
 ('xcqntej6',
  'finetune_v1_finetune_ETTm1_mask_25_FRZN_2024-03-06_00:40:54.241935'),
 ('9tmxet4e',
  'finetune_v1_finetune_ETTm1_mask_50_FRZN_2024-03-06_00:53:08.502292'),
 ('50g3mql7',
  'finetune_v1_finetune_ETTm1_mask_75_FRZN_2024-03-06_01:05:04.795560'),
 ('t9z61ui4',
  'finetune_v1_finetune_ETTm1_mask_25_NOT_FRZN_2024-03-06_01:17:18.409522'),
 ('2urlsb9v',
  'finetune_v1_finetune_ETTm1_mask_50_NOT_FRZN_2024-03-06_01:30:20.148318'),
 ('j9ly5app',
  'finetune_v1_finetune_ETTm1_mask_75_NOT_FRZN_2024-03-06_01:43:36.635241')]

In [20]:
ettm2

[('1pqd5l5t', 'pretrain_v1_pretrain_ETTm2_mask_25_2024-03-05_23:10:11.573805'),
 ('173yqu26', 'pretrain_v1_pretrain_ETTm2_mask_50_2024-03-06_00:06:49.304020'),
 ('pops45gk', 'pretrain_v1_pretrain_ETTm2_mask_75_2024-03-06_00:52:11.877415'),
 ('zg8l4nbw',
  'finetune_v1_finetune_ETTm2_mask_25_FRZN_2024-03-06_01:40:01.422074'),
 ('3jeq1g35',
  'finetune_v1_finetune_ETTm2_mask_50_FRZN_2024-03-06_02:02:03.669746'),
 ('795ruzmk',
  'finetune_v1_finetune_ETTm2_mask_75_FRZN_2024-03-06_02:15:09.575722'),
 ('24qaofj2',
  'finetune_v1_finetune_ETTm2_mask_25_NOT_FRZN_2024-03-06_02:28:09.632661'),
 ('t1hpy3i5',
  'finetune_v1_finetune_ETTm2_mask_50_NOT_FRZN_2024-03-06_02:41:51.728101'),
 ('9wk2osze',
  'finetune_v1_finetune_ETTm2_mask_75_NOT_FRZN_2024-03-06_02:55:31.803292')]

In [21]:
finetune_cols = ['Finetune25_Frozen |', '| Finetune50_Frozen |', '| Finetune75_Frozen |', '| Finetune25_NF |', '| Finetune50_NF |', '| Finetune75_NF |']
pretrain_cols = ['Pretrain_25 |', '| Pretrain_50 |', '| Pretrain_75 |']

In [24]:
def highlight_min(s):
    is_min = s == s.min()
    # styles = ['background-color: yellow' if v else '' for v in is_min]
    text_color = ['color: red' if v else '' for v in is_min]
    return text_color

def dataset_runs(dataset_ls):
    summaries_pretrain = []
    summaries_finetune = []
    
    for run_id, name in dataset_ls:
        run = api.run(f"aneog/ett/{run_id}")

        summary = run.summary
        params = run.config

        if 'finetune' in name:
            print(f"Summary for Run {name}:")
            summary_dict_val = {}
            summary_dict_train = {}
            summary_dict_test = {}
            
            for key_val in finetune_val_features:
                if key_val in summary:
                    summary_dict_val[key_val] = summary[key_val]
                else:
                    print(f"Finetune: Key: {key_val} not found in the dictionary.")
            
            for key_train in finetune_train_features:
                if key_train in summary:
                    summary_dict_train[key_train] = summary[key_train]
                else:
                    print(f"Finetune: Key: {key_train} not found in the dictionary.")
                    
            for key_test in finetune_test_features:
                if key_test in summary:
                    summary_dict_test[key_test] = summary[key_test]
                else:
                    print(f"Finetune: Key: {key_test} not found in the dictionary.")
                    
            summaries_finetune.append(list(summary_dict_test.values()) + list(summary_dict_val.values()) + list(summary_dict_train.values()))
            
        if 'pretrain' in name:
            print(f"Summary for Run {name}:")
            summary_val_dict = {}
            summary_test_dict = {}
            
            for key in pretrain_val_features:
                if key in summary:
                    summary_val_dict[key] = summary[key]
                else:
                    print(f"Pretrain: Key: {key} not found in the dictionary.")
                    
            for key in pretrain_test_features:
                if key in summary:
                    summary_test_dict[key] = summary[key]
                else:
                    print(f"Pretrain: Key: {key} not found in the dictionary.")
                    
            summaries_pretrain.append(list(summary_test_dict.values()) + list(summary_val_dict.values()))
    
    if len(summaries_finetune)!=0:
        finetune_df = pd.DataFrame(summaries_finetune, columns=finetune_test_features + finetune_val_features + finetune_train_features).transpose()
        finetune_df.columns = finetune_cols
    
    if len(summaries_pretrain)!=0:
        pretrain_df = pd.DataFrame(summaries_pretrain, columns=pretrain_test_features + pretrain_val_features).transpose()
        pretrain_df.columns = pretrain_cols
        
    if len(summaries_pretrain)!=0 and len(summaries_finetune)!=0:
        return pretrain_df.style.apply(highlight_min, axis=1), finetune_df.style.apply(highlight_min, axis=1)
    elif len(summaries_pretrain)!=0:
        return pretrain_df.style.apply(highlight_min, axis=1), None
    else:
        return None, finetune_df.style.apply(highlight_min, axis=1)

In [25]:
etth1_pretrain, etth1_finetune = dataset_runs(etth1)

Summary for Run pretrain_v1_pretrain_ETTh1_mask_25_2024-03-05_23:09:41.446873:
Summary for Run pretrain_v1_pretrain_ETTh1_mask_50_2024-03-05_23:24:39.183368:
Summary for Run pretrain_v1_pretrain_ETTh1_mask_75_2024-03-05_23:52:07.764429:
Summary for Run finetune_v1_finetune_ETTh1_mask_25_FRZN_2024-03-06_00:15:16.376254:
Summary for Run finetune_v1_finetune_ETTh1_mask_50_FRZN_2024-03-06_01:08:01.749827:
Summary for Run finetune_v1_finetune_ETTh1_mask_75_FRZN_2024-03-06_02:01:58.181763:
Summary for Run finetune_v1_finetune_ETTh1_mask_25_NOT_FRZN_2024-03-06_02:30:02.259881:
Summary for Run finetune_v1_finetune_ETTh1_mask_50_NOT_FRZN_2024-03-06_02:50:59.879849:
Summary for Run finetune_v1_finetune_ETTh1_mask_75_NOT_FRZN_2024-03-06_03:07:15.656449:


In [26]:
etth2_pretrain, etth2_finetune = dataset_runs(etth2)

Summary for Run pretrain_v1_pretrain_ETTh2_mask_25_2024-03-05_23:09:50.208599:
Summary for Run pretrain_v1_pretrain_ETTh2_mask_50_2024-03-05_23:20:15.457856:
Summary for Run pretrain_v1_pretrain_ETTh2_mask_75_2024-03-05_23:34:39.283505:
Summary for Run finetune_v1_finetune_ETTh2_mask_25_FRZN_2024-03-05_23:47:05.142523:
Summary for Run finetune_v1_finetune_ETTh2_mask_50_FRZN_2024-03-06_00:16:14.255934:
Summary for Run finetune_v1_finetune_ETTh2_mask_75_FRZN_2024-03-06_00:44:31.044504:
Summary for Run finetune_v1_finetune_ETTh2_mask_25_NOT_FRZN_2024-03-06_01:13:49.788276:
Summary for Run finetune_v1_finetune_ETTh2_mask_50_NOT_FRZN_2024-03-06_01:42:45.253870:
Summary for Run finetune_v1_finetune_ETTh2_mask_75_NOT_FRZN_2024-03-06_02:09:53.986500:


In [27]:
ettm1_pretrain, ettm1_finetune = dataset_runs(ettm1)

Summary for Run pretrain_v1_pretrain_ETTm1_mask_25_2024-03-05_23:10:01.710980:
Summary for Run pretrain_v1_pretrain_ETTm1_mask_50_2024-03-05_23:41:47.157029:
Summary for Run pretrain_v1_pretrain_ETTm1_mask_75_2024-03-06_00:11:46.385247:
Summary for Run finetune_v1_finetune_ETTm1_mask_25_FRZN_2024-03-06_00:40:54.241935:
Summary for Run finetune_v1_finetune_ETTm1_mask_50_FRZN_2024-03-06_00:53:08.502292:
Summary for Run finetune_v1_finetune_ETTm1_mask_75_FRZN_2024-03-06_01:05:04.795560:
Summary for Run finetune_v1_finetune_ETTm1_mask_25_NOT_FRZN_2024-03-06_01:17:18.409522:
Summary for Run finetune_v1_finetune_ETTm1_mask_50_NOT_FRZN_2024-03-06_01:30:20.148318:
Summary for Run finetune_v1_finetune_ETTm1_mask_75_NOT_FRZN_2024-03-06_01:43:36.635241:


In [28]:
ettm2_pretrain, ettm2_finetune = dataset_runs(ettm2)

Summary for Run pretrain_v1_pretrain_ETTm2_mask_25_2024-03-05_23:10:11.573805:
Summary for Run pretrain_v1_pretrain_ETTm2_mask_50_2024-03-06_00:06:49.304020:
Summary for Run pretrain_v1_pretrain_ETTm2_mask_75_2024-03-06_00:52:11.877415:
Summary for Run finetune_v1_finetune_ETTm2_mask_25_FRZN_2024-03-06_01:40:01.422074:
Summary for Run finetune_v1_finetune_ETTm2_mask_50_FRZN_2024-03-06_02:02:03.669746:
Summary for Run finetune_v1_finetune_ETTm2_mask_75_FRZN_2024-03-06_02:15:09.575722:
Summary for Run finetune_v1_finetune_ETTm2_mask_25_NOT_FRZN_2024-03-06_02:28:09.632661:
Summary for Run finetune_v1_finetune_ETTm2_mask_50_NOT_FRZN_2024-03-06_02:41:51.728101:
Summary for Run finetune_v1_finetune_ETTm2_mask_75_NOT_FRZN_2024-03-06_02:55:31.803292:


In [29]:
legend = """
    <h3>Legend:</h3>
    <ul>
        <li>Red text: Lowest RMSE value in each row</li>
    </ul>
    <ul>
        <li>Finetune25_FROZEN: 25% Masked Pretrained model finetuned with a Frozen encoder
    </ul>
    <ul>
        <li>Finetune75_NF: 75% Masked Pretrained model Fully Fine-Tuned (i.e. encoder not frozen)
    </ul>
    <br>
    <h3>Training Procedure:</h3>
    <ul>
        <li> Data was split in the ratio = 6:2:2 (standard for ETT)
        <li> Lookback Window = 336 timesteps
        <li> Horizon Window = 96 timesteps
        <li> Pre-trained epochs = 30
        <li> Fine-tune epochs = 50
        <li> 3 Models were pretrained on 60% of each dataset with following mask ratios: 25, 50, 75
        <li> Once pretrained, each model was used to finetune (or applied directly) on each dataset (60% portion of it) and tested on the 20%
    </ul>
    """

with open('ett_results.html', 'w') as f:
    f.write('<html><body>')
    f.write(legend)
    # f.write('<h2> ETTh1 Pre-train </h2>')
    # f.write(combined_pretrain.render())
    
    f.write('<h2> ETTh1 </h2>')
    f.write(etth1_finetune.render())
    
    f.write('<h2> ETTh2 </h2>')
    f.write(etth2_finetune.render())
    
    f.write('<h2> ETTm1 </h2>')
    f.write(ettm1_finetune.render())
    
    f.write('<h2> ETTm2 </h2>')
    f.write(ettm2_finetune.render())
    
#     f.write('<h2> PRLA Observed Downstream </h2>')
#     f.write(prla_obs_finetune.render())
    
#     f.write('<h2> PRPO Observed Downstream </h2>')
#     f.write(prpo_obs_finetune.render())
    
#     f.write('<h2> SUGG Observed Downstream </h2>')
#     f.write(sugg_obs_finetune.render())
    
#     f.write('<h2> TOOK Observed Downstream </h2>')
#     f.write(took_obs_finetune.render())
    
    
    f.write('</body></html>')

/tmp/ipykernel_30188/1068037491.py:32: FutureWarning: this method is deprecated in favour of `Styler.to_html()`
  f.write(etth1_finetune.render())
/tmp/ipykernel_30188/1068037491.py:35: FutureWarning: this method is deprecated in favour of `Styler.to_html()`
  f.write(etth2_finetune.render())
/tmp/ipykernel_30188/1068037491.py:38: FutureWarning: this method is deprecated in favour of `Styler.to_html()`
  f.write(ettm1_finetune.render())
/tmp/ipykernel_30188/1068037491.py:41: FutureWarning: this method is deprecated in favour of `Styler.to_html()`
  f.write(ettm2_finetune.render())
